In [ ]:
### 문정역 근처 top5 아파트 시세
# 올림픽 훼미리, 가락한신, 문정한전현대, 문정푸르지오1차, 건영

In [3]:
import requests
import json
from bs4 import BeautifulSoup
import time
import pandas as pd

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time

In [5]:
## 복호화함수
import base64
from Crypto.Cipher import AES
from Crypto.Util.Padding import unpad



def decrypt(encrypted_text, secret):
    """
    AES ECB 모드로 암호화된 텍스트를 복호화합니다.
    
    Args:
        encrypted_text (str): Base64로 인코딩된 암호화된 텍스트
    
    Returns:
        str: 복호화된 평문 텍스트
    """
    # Base64 디코딩
    encrypted_bytes = base64.b64decode(encrypted_text)
    
    # 키 생성
    key = get_key(secret)
    
    # AES 복호화 객체 생성 (ECB 모드)
    cipher = AES.new(key, AES.MODE_ECB)
    
    # 복호화
    decrypted_bytes = cipher.decrypt(encrypted_bytes)
    
    # PKCS7 패딩 제거
    unpadded_bytes = unpad(decrypted_bytes, AES.block_size)
    
    # UTF-8 문자열로 변환
    return unpadded_bytes.decode('utf-8')

def get_key(secret):
    """
    비밀 문자열을 AES 키로 변환합니다.
    키 길이에 따라 적절한 크기로 조정합니다.
    
    Args:
        secret (str): 비밀 문자열
    
    Returns:
        bytes: AES 키 (16, 24, 또는 32 바이트)
    """
    # 문자열을 UTF-8 바이트로 변환
    key_bytes = secret.encode('utf-8')
    length = len(key_bytes)
    
    # 키 길이에 따라 조정
    if length < 16:
        # 16바이트보다 작으면 16바이트로 패딩
        adjusted_key = key_bytes[:16].ljust(16, b'\x00')
    elif length > 16 and length < 24:
        # 16바이트보다 크고 24바이트보다 작으면 24바이트로 조정
        adjusted_key = key_bytes[:24].ljust(24, b'\x00')
    elif length > 24 and length < 32:
        # 24바이트보다 크고 32바이트보다 작으면 32바이트로 조정
        adjusted_key = key_bytes[:32].ljust(32, b'\x00')
    elif length >= 32:
        # 32바이트 이상이면 32바이트로 잘라내기
        adjusted_key = key_bytes[:32]
    else:
        # 정확히 16바이트면 그대로 사용
        adjusted_key = key_bytes
    
    return adjusted_key

In [7]:
# 억단위 숫자로
import re

def parse_korean_price(text):
    """
    '1억 2,300' → 123000000 변환
    """
    if pd.isna(text):
        return None
    text = text.replace(',', '').replace(' ', '')
    total = 0
    match = re.match(r'(?:(\d+)억)?(?:(\d+)천)?(\d+)?', text)
    if not match:
        return None
    if match.group(1):  # 억
        total += int(match.group(1)) * 100_000_000
    if match.group(2):  # 천만 단위
        total += int(match.group(2)) * 10_000_000
    if match.group(3):  # 만 단위 이하
        total += int(match.group(3))
    return total

In [8]:
## 올림픽 훼미리
import requests
import re

# apt_info.jsp
url = "https://asil.kr/app/apt_info.jsp?os=pc&apt=2287"
headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://asil.kr/",
}
response = requests.get(url, headers=headers)

# HTML 본문
html = response.text

# 정규표현식으로 getKey("숫자")에서 숫자만 문자열로 추출
keys = re.findall(r'getKey\("(\d+)"\)', html)

# 결과 확인
print(keys)

['7573695431252582287']


In [12]:
## 올림픽 훼미리
import requests
import pandas as pd


url = ("https://asil.kr/app/data/apt_price_m2_newver_6.jsp?sido=11&dealmode=1&building=apt&seq=2287&m2=0&py=&py_type=&isPyQuery=false&year=9999&u=0&start=0&count=10000&dong_name=&order=")

headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://asil.kr/",
    "X-Requested-With": "XMLHttpRequest",
    "Accept": "application/json"
}

response = requests.get(url, headers=headers)

try:
    new_data = response.json()
except ValueError:
    print("JSON 디코딩 실패. 응답 내용:")
    print(response.text)
    exit()

rows = []

if new_data and "val" in new_data[0]:
    for monthly in new_data[0]["val"]:
        yyyymm = monthly["yyyymm"]
        for daily in monthly["val"]:
            day = daily["day"]
            for deal in daily["val"]:
                rows.append({
                    "거래일": f"{yyyymm[:4]}.{yyyymm[4:]}.{day:02}",
                    "층수": deal.get("floor"),
                    "가격": deal.get("money"),
                    "전월세": deal.get("rent")
                })
else:
    print("빈 데이터 혹은 예상 구조와 다름")
    exit()

df_1 = pd.DataFrame(rows)


key = '7573695431252582287'

# 가격과 전월세 컬럼 복호화
df_1['가격'] = df_1['가격'].apply(lambda x: decrypt(x, key))
df_1['전월세'] = df_1['전월세'].apply(lambda x: decrypt(x, key))

df_1['가격(숫자)'] = df_1['가격'].apply(parse_korean_price)  # parse_korean_price는 이전에 정의한 함수
df_1


,거래일,층수,가격,전월세,가격(숫자)
0,2025.07.14,5,"27억 5,000",,2700005000
1,2025.07.12,2,27억,,2700000000
2,2025.07.12,3,"27억 6,000",,2700006000
3,2025.07.02,14,28억,,2800000000
4,2025.07.01,5,"26억 5,000",,2600005000
...,...,...,...,...,...
3267,2006.01.06,6,"7억 4,000",,700004000
3268,2006.01.06,8,"7억 4,000",,700004000
3269,2006.01.05,14,"7억 2,000",,700002000
3270,2006.01.03,12,"6억 9,000",,600009000


In [7]:
## 가락한신
import requests
import re

# apt_info.jsp
url = "https://asil.kr/app/apt_info.jsp?os=pc&apt=2267"
headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://asil.kr/",
}
response = requests.get(url, headers=headers)

# HTML 본문
html = response.text

# 정규표현식으로 getKey("숫자")에서 숫자만 문자열로 추출
keys = re.findall(r'getKey\("(\d+)"\)', html)

# 결과 확인
print(keys)

['7573695431259982267']


In [8]:
## 가락한신
import requests
import pandas as pd


url = ("https://asil.kr/app/data/apt_price_m2_newver_6.jsp?sido=11&dealmode=1&building=apt&seq=2267&m2=0&py=&py_type=&isPyQuery=false&year=9999&u=0&start=0&count=10000&dong_name=&order=")

headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://asil.kr/",
    "X-Requested-With": "XMLHttpRequest",
    "Accept": "application/json"
}

response = requests.get(url, headers=headers)

try:
    new_data = response.json()
except ValueError:
    print("JSON 디코딩 실패. 응답 내용:")
    print(response.text)
    exit()

rows = []

if new_data and "val" in new_data[0]:
    for monthly in new_data[0]["val"]:
        yyyymm = monthly["yyyymm"]
        for daily in monthly["val"]:
            day = daily["day"]
            for deal in daily["val"]:
                rows.append({
                    "거래일": f"{yyyymm[:4]}.{yyyymm[4:]}.{day:02}",
                    "층수": deal.get("floor"),
                    "가격": deal.get("money"),
                    "전월세": deal.get("rent")
                })
else:
    print("빈 데이터 혹은 예상 구조와 다름")
    exit()

df_2 = pd.DataFrame(rows)

key = '7573695431259982267'

# 가격과 전월세 컬럼 복호화
df_2['가격'] = df_2['가격'].apply(lambda x: decrypt(x, key))
df_2['전월세'] = df_2['전월세'].apply(lambda x: decrypt(x, key))

df_2['가격(숫자)'] = df_2['가격'].apply(parse_korean_price)  # parse_korean_price는 이전에 정의한 함수
df_2

,거래일,층수,가격,전월세,가격(숫자)
0,2025.07.07,8,"12억 9,500",,1200009500
1,2025.07.04,11,"12억 5,000",,1200005000
2,2025.03.08,10,12억,,1200000000
3,2025.03.08,10,12억,,1200000000
4,2025.02.15,6,11억,,1100000000
...,...,...,...,...,...
215,2006.04.29,3,"5억 2,000",,500002000
216,2006.04.23,3,"5억 1,500",,500001500
217,2006.03.24,11,"5억 2,500",,500002500
218,2006.03.14,9,"5억 2,300",,500002300


In [9]:
## 문정한전현대
import requests
import re

# apt_info.jsp
url = "https://asil.kr/app/apt_info.jsp?os=pc&apt=2284"
headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://asil.kr/",
}
response = requests.get(url, headers=headers)

# HTML 본문
html = response.text

# 정규표현식으로 getKey("숫자")에서 숫자만 문자열로 추출
keys = re.findall(r'getKey\("(\d+)"\)', html)

# 결과 확인
print(keys)

['7543695431253692284']


In [10]:
## 문정한전현대
import requests
import pandas as pd


url = ("https://asil.kr/app/data/apt_price_m2_newver_6.jsp?sido=11&dealmode=1&building=apt&seq=2284&m2=0&py=&py_type=&isPyQuery=false&year=9999&u=0&start=0&count=10000&dong_name=&order=")

headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://asil.kr/",
    "X-Requested-With": "XMLHttpRequest",
    "Accept": "application/json"
}

response = requests.get(url, headers=headers)

try:
    new_data = response.json()
except ValueError:
    print("JSON 디코딩 실패. 응답 내용:")
    print(response.text)
    exit()

rows = []

if new_data and "val" in new_data[0]:
    for monthly in new_data[0]["val"]:
        yyyymm = monthly["yyyymm"]
        for daily in monthly["val"]:
            day = daily["day"]
            for deal in daily["val"]:
                rows.append({
                    "거래일": f"{yyyymm[:4]}.{yyyymm[4:]}.{day:02}",
                    "층수": deal.get("floor"),
                    "가격": deal.get("money"),
                    "전월세": deal.get("rent")
                })
else:
    print("빈 데이터 혹은 예상 구조와 다름")
    exit()

df_3 = pd.DataFrame(rows)

key = '7543695431253692284'

# 가격과 전월세 컬럼 복호화
df_3['가격'] = df_3['가격'].apply(lambda x: decrypt(x, key))
df_3['전월세'] = df_3['전월세'].apply(lambda x: decrypt(x, key))

df_3['가격(숫자)'] = df_3['가격'].apply(parse_korean_price)  # parse_korean_price는 이전에 정의한 함수
df_3

,거래일,층수,가격,전월세,가격(숫자)
0,2025.06.12,1,"7억 9,900",,700009900
1,2025.06.10,3,"7억 1,000",,700001000
2,2025.04.15,4,"8억 2,000",,800002000
3,2025.04.12,7,"8억 5,000",,800005000
4,2025.03.23,6,9억,,900000000
...,...,...,...,...,...
281,2006.04.06,8,4억,,400000000
282,2006.03.21,9,"2억 7,700",,200007700
283,2006.03.05,5,"2억 6,000",,200006000
284,2006.01.23,14,"2억 4,750",,200004750


In [11]:
## 문정푸르지오 1차
import requests
import re

# apt_info.jsp
url = "https://asil.kr/app/apt_info.jsp?os=pc&apt=2278"
headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://asil.kr/",
}
response = requests.get(url, headers=headers)

# HTML 본문
html = response.text

# 정규표현식으로 getKey("숫자")에서 숫자만 문자열로 추출
keys = re.findall(r'getKey\("(\d+)"\)', html)

# 결과 확인
print(keys)

['7583695431255912278']


In [13]:
## 문정푸르지오 1차
import requests
import pandas as pd


url = ("https://asil.kr/app/data/apt_price_m2_newver_6.jsp?sido=11&dealmode=1&building=apt&seq=2278&m2=0&py=&py_type=&isPyQuery=false&year=9999&u=0&start=0&count=1000&dong_name=&order=")

headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://asil.kr/",
    "X-Requested-With": "XMLHttpRequest",
    "Accept": "application/json"
}

response = requests.get(url, headers=headers)

try:
    new_data = response.json()
except ValueError:
    print("JSON 디코딩 실패. 응답 내용:")
    print(response.text)
    exit()

rows = []

if new_data and "val" in new_data[0]:
    for monthly in new_data[0]["val"]:
        yyyymm = monthly["yyyymm"]
        for daily in monthly["val"]:
            day = daily["day"]
            for deal in daily["val"]:
                rows.append({
                    "거래일": f"{yyyymm[:4]}.{yyyymm[4:]}.{day:02}",
                    "층수": deal.get("floor"),
                    "가격": deal.get("money"),
                    "전월세": deal.get("rent")
                })
else:
    print("빈 데이터 혹은 예상 구조와 다름")
    exit()

df_4 = pd.DataFrame(rows)

key = '7583695431255912278'

# 가격과 전월세 컬럼 복호화
df_4['가격'] = df_4['가격'].apply(lambda x: decrypt(x, key))
df_4['전월세'] = df_4['전월세'].apply(lambda x: decrypt(x, key))

df_4['가격(숫자)'] = df_4['가격'].apply(parse_korean_price)  # parse_korean_price는 이전에 정의한 함수
df_4

,거래일,층수,가격,전월세,가격(숫자)
0,2025.07.05,8,"11억 1,000",,1100001000
1,2025.06.28,6,"10억 4,800",,1000004800
2,2025.06.27,11,"13억 2,000",,1300002000
3,2025.06.22,14,"13억 3,000",,1300003000
4,2025.05.31,11,"10억 9,500",,1000009500
...,...,...,...,...,...
274,2006.04.13,10,"5억 7,800",,500007800
275,2006.04.10,7,"5억 6,000",,500006000
276,2006.03.22,11,"3억 6,300",,300006300
277,2006.03.04,9,"5억 5,700",,500005700


In [14]:
## 건영
import requests
import re

# apt_info.jsp
url = "https://asil.kr/app/apt_info.jsp?os=pc&apt=2276"
headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://asil.kr/",
}
response = requests.get(url, headers=headers)

# HTML 본문
html = response.text

# 정규표현식으로 getKey("숫자")에서 숫자만 문자열로 추출
keys = re.findall(r'getKey\("(\d+)"\)', html)

# 결과 확인
print(keys)

['7563695431256652276']


In [16]:
## 건영
import requests
import pandas as pd


url = ("https://asil.kr/app/data/apt_price_m2_newver_6.jsp?sido=11&dealmode=1&building=apt&seq=2276&m2=0&py=&py_type=&isPyQuery=false&year=9999&u=0&start=0&count=1000&dong_name=&order=")

headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://asil.kr/",
    "X-Requested-With": "XMLHttpRequest",
    "Accept": "application/json"
}

response = requests.get(url, headers=headers)

try:
    new_data = response.json()
except ValueError:
    print("JSON 디코딩 실패. 응답 내용:")
    print(response.text)
    exit()

rows = []

if new_data and "val" in new_data[0]:
    for monthly in new_data[0]["val"]:
        yyyymm = monthly["yyyymm"]
        for daily in monthly["val"]:
            day = daily["day"]
            for deal in daily["val"]:
                rows.append({
                    "거래일": f"{yyyymm[:4]}.{yyyymm[4:]}.{day:02}",
                    "층수": deal.get("floor"),
                    "가격": deal.get("money"),
                    "전월세": deal.get("rent")
                })
else:
    print("빈 데이터 혹은 예상 구조와 다름")
    exit()

df_5 = pd.DataFrame(rows)

key = '7563695431256652276'

# 가격과 전월세 컬럼 복호화
df_5['가격'] = df_5['가격'].apply(lambda x: decrypt(x, key))
df_5['전월세'] = df_5['전월세'].apply(lambda x: decrypt(x, key))

df_5['가격(숫자)'] = df_5['가격'].apply(parse_korean_price)  # parse_korean_price는 이전에 정의한 함수
df_5

,거래일,층수,가격,전월세,가격(숫자)
0,2025.07.01,6,"12억 5,000",,1200005000
1,2025.05.10,7,13억,,1300000000
2,2025.04.26,15,"12억 4,000",,1200004000
3,2025.03.21,6,"11억 9,000",,1100009000
4,2025.03.17,3,"12억 2,000",,1200002000
...,...,...,...,...,...
462,2006.02.11,6,"4억 9,000",,400009000
463,2006.02.08,7,"5억 4,500",,500004500
464,2006.02.03,8,"5억 1,000",,500001000
465,2006.02.01,7,5억 500,,500000500
